<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/Document_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install konlpy
!pip3 install jamo

In [2]:
import requests # 파파고 번역 api 사용
import re
import jpype
import requests
from gensim.models import Word2Vec
from jamo import h2j, j2hcj
from konlpy.tag import Okt
import pandas as pd

In [3]:
# 구글 드라이브 연결
import os
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/PMS/"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [4]:
# 파일 확장자 및 특수문자 제거 후 형태소 명사 단위로 분석

def morpheme(file_name):
  okt = Okt()

  no_extension_file_name = file_name.rsplit('.', 1)[0] 
  cleaned_file_name = re.sub('[-=+,_#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',
                             '', no_extension_file_name)

  return cleaned_file_name

In [5]:
# 영어 문장 한글로 번역

def get_translate_ko(text):
    #client_id = "36sG5L0c4JVanssDM_3E" # <-- client_id 기입
    #client_secret = "nJ4nIqNdgh" # <-- client_secret 기입
    client_id = "yays8a0UDCVyw8yb6KzY" # <-- client_id 기입
    client_secret = "E5gVbYEht8" # <-- client_secret 기입

    data = {'text' : text,
            'source' : 'en',
            'target': 'ko'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)

In [6]:
# 한글 문장 영어로 번역

def get_translate_eng(text):
    client_id = "fSuwQC7wzKdKWD4f0Gbl" # <-- client_id 기입
    client_secret = "upCLLCyDRI" # <-- client_secret 기입

    data = {'text' : text,
            'source' : 'ko',
            'target': 'en'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)

In [7]:
# 데이터 전처리 함수

def data_preprocess(data_list): # 파일 리스트 / 디렉토리 리스트
    preprocessed_list = data_list
    tmp_data=""
    result = []

    # 형태소 분석, 특수기호 제거
    for data_index in range(len(preprocessed_list)):
        split_data = morpheme(preprocessed_list[data_index])
        preprocessed_list[data_index] = split_data
    
    # 번역 api : 영어 번역, 띄어쓰기
    for index in range(len(preprocessed_list)):
      # 영어 거쳐서 한국어 번역하면 정확도 높은 대신
      # 시간 오래걸리고 api 일일 사용량 제한 더 잘걸림
      eng_list = get_translate_eng(preprocessed_list[index])
      ko_list = get_translate_ko(eng_list)
      # ko_list = get_translate_ko(preprocessed_list[index])
      result.append(ko_list)
    return result

In [8]:
# 폴더 안에 들어간 파일들의 내용 저장

def save_classificated_file(index_dict, file_list, directory_list):
    result_dict = {}

    for key, value in index_dict.items():
        result_dict[file_list[key]] = directory_list[value]

    return result_dict

In [9]:
# 매칭된 파일 제거

def rm_classificated_file(index_dict,file_list):  # 매칭된 파일 제거
    data_name = []
    for index in list(index_dict.keys()):
        data_name.append(file_list[index])

    file_list =  [x for x in file_list if x not in (data_name)]

    return file_list

In [10]:
# 단순 유사도 검사
# 파일명과 카테고리명이 같으면 그 폴더 안에 넣어줌

def simplest_classification(directory_list, file_list):  # 단어 유사도 검사 => index return해서 제거하는 방법을 쓴다.
    tmp_dict = {}
    if jpype.isJVMStarted():
      jpype.attachThreadToJVM()
    okt = Okt()
    classificated_dir_index = 0
    for file_index in range(len(file_list)):
        file_morphs = okt.morphs(file_list[file_index])
        
        max_weight = 0
        for dir_index in range(len(directory_list)): # 디렉토리 갯수 : 3
            dir_morphs = okt.morphs(directory_list[dir_index])
            tmp_weight = 0
            for file_element in file_morphs: # 형태분석한 파일 갯수
                for dir_element in dir_morphs:
                    
                    if (dir_element == file_element) and (check_format(dir_element) == ('kor')):
                        tmp_weight += 2      
                    	#한글로 되있을 경우 가중치 2
                    elif ((file_element in dir_element) and (check_format(dir_element) == ('num'))):
                        tmp_weight += 2
                        #영어로 되있을 경우 가중치 1
                        
            if (tmp_weight > max_weight):
                max_weight = tmp_weight
                classificated_dir_index = dir_index
        if max_weight > 0:
            tmp_dict[file_index] = classificated_dir_index

    return tmp_dict

In [11]:
# 한글과 숫자를 판단하고 가중치를 넣어주는 함수
# 숫자보다 한글의 가중치가 높게 설정

def check_format(input): # 한국어 , 영어 , 숫자 형태 분석
    value = input[0]
    if ord('가') <= ord(value) <= ord('힣'):
        return 'kor'
    elif value.isdigit():
        return 'num'
    elif value.isalpha():
        return 'eng'
    else:
        return False

In [12]:
# Word2Vec 유사도 측정
# 제목과 카테고리명 사이의 유사도를 측정, 퍼센테이지 출력

def word2vec_similarity(directory_list, file_list):
    model = Word2Vec.load(path+'ko.bin')

    tmp_dict = {}
    classificated_dir_index = 0
    if jpype.isJVMStarted():
      jpype.attachThreadToJVM()
    okt = Okt() # 형태소 분석
    for file_index in range(len(file_list)):
        max_similarity = 0
        file_morphs = okt.morphs(file_list[file_index])
        for dir_index in range(len(directory_list)):
            dir_morphs = okt.morphs(directory_list[dir_index])
            tmp_similarity = 0
            for file_element in file_morphs:
                for dir_element in dir_morphs:
                    if check_format(file_element) == 'kor' and check_format(dir_element) == 'kor':
                        try:
                            tmp_similarity = model.similarity(file_element, dir_element)
                        except KeyError:
                            pass
                if tmp_similarity > max_similarity:  # 유사도 수치는 사용자가 설정
                    max_similarity = tmp_similarity
                    classificated_dir_index = dir_index

            tmp_dict[file_index] = classificated_dir_index

    return tmp_dict

In [13]:
# 자모 단위를 이용한 유사도 검사

def Phonology_classification(directory_list, file_list):
    trans_file_jumo = []
    trans_folder_jumo = []
    equl_count = [[0 for i in range(len(file_list))] for i in range(len(directory_list))]
    result_dict = {}

    for i in file_list:
        a = j2hcj(h2j(i))
        line = []
        for j in a:
            line.append(j2hcj(h2j(j)))
        trans_file_jumo.append(line)

    for i in directory_list:
        a = j2hcj(h2j(i))
        line = []
        for j in a:
            line.append(j2hcj(h2j(j)))
        trans_folder_jumo.append(line)

    for i in range(len(directory_list)):
        for j in range(len(file_list)):
            for k in trans_file_jumo[j]:
                if k in trans_folder_jumo[i]:
                    equl_count[i][j] += 1

    for i in range(len(directory_list)):
        for j in range(len(file_list)):
            if len(trans_folder_jumo[i]) <= equl_count[i][j]:
                result_dict[j] = i

    return result_dict

In [14]:
def dict_to_list(dict):
    result = []

    for key, value in dict.items():
        tmp = str(value) + "/"+ str(key)
        result.append(tmp)

    return result

In [15]:
# 기타 폴더
# 분류가 안된 파일들을 기타로 분류해줌

def except_directory(file_list):
    out_of_classification = "except_directory"
    result_dict = {}

    for file in file_list:
        result_dict[file] = out_of_classification

    return result_dict

In [25]:
# 분류될 카테고리명
directory_list = ['영어', '외국어', '국어', '문예', '네이밍', '슬로건', '논문', '수학', '과학', '학습•문예', '코딩', '프로그래밍', '해커톤', '메이커톤', '해킹', '보안', '빅데이터', '인공지능', '사물인터넷', 'IT', '컴퓨터', '달리기', '마라톤', '자전거', '수영', '야구', '축구', '농구', '배구', '배드민턴', '테니스', '골프', '볼링', '당구', '탁구 ', '무예', '레포츠', 'e-스포츠', '스포츠', '그림', '그리기', '디자인', '캐릭터', '웹툰', '공예', '만들기', '미술', '동요', '성악', '실용음악', '가요', '국악', '무용', '발레', '댄스', '배우', '오디션', '음악', '무용', '배우', '사진', 'UCC', '동영상', '드론', '요리', '반려동물', '뷰티', '선발', '주산암산', '바둑', '체스', '건축', '제안', '아이디어', '산업', '사회', '창업', '퀴즈', '이색']
# 분류할 문서 이름
file_list = []
data = pd.read_csv(path+"data2.csv", encoding="cp949")
for i in range(len(data)):
  file_list.append(data['Column1'][i])
file_list = file_list[400:430]
print(file_list)

['평택시 신중년의 손 사진 공모전 「인생을 담다」', '2021년도 부산관광공사 시민 아이디어 공모전', '제9회 아름다운 우리 국토 사진공모전', '치매예방 시니어 가요 콩쿨', '제4회 전국 중학생 K-POP 경연대회', '제5회 GO BACK 예산장터 삼국축제 공모전', "제3회 시민참여 공모전-'쓰.담 공모전' 일터생활 공모전", '탄소중립 달성을 위한 전력산업 아이디어 발굴 경진대회', '<2021 서점의 날> 지역서점 브이로그 공모전', '2021년 삼각산 동요대회', '2021년 나마스떼코리아 제6회 히말라야 사진(영상) 공모전 [기간연장]', '제주특별자치도 청년 캐릭터 공모전', '2021 전국 학생발명상상화 및 캐릭터 · 디자인 그리기 대회', '2021 군산시 슬기로운 가족생활 사진공모전', '제18회 청소년 봉사활동 실천사례 발표대회', '석유 데이터 활용 아이디어 공모전', '2021 SK하이닉스 기후변화 위기극복 작품공모전', '제12회 통신서비스 이용자주간 통신서비스 피해예방 웹툰공모전', '제1회 수목원·정원 분야 국민참여 아이디어 공모전', '국민안전 119초 영상 공모전', '2022 제주들불축제 콘텐츠 아이디어 공모전', '2021년도 한국기술교육대학교 안전 분야 혁신 아이디어 공모전', '증평인삼문화센터 BI 디자인 공모전', '모인 해외송금 학생 할인 서비스 네이밍 공모전', '필환경 제주관광 슬로건 공모전', '제5회 대학생 통상정책 토론대회', '다문화 이주여성 대상 한국산 임산물 활용 현지 요리 UCC 공모전', '2021 대한민국 교육 공로 표창 행사(공모전)', '청풍아름그린 사진 & 100초영상 공모전', '2021 수출 핫템 공모전']


In [26]:
result_dict = {}
index_dict = {}

origin_directory_list = directory_list.copy()
origin_file_list = file_list.copy()

In [27]:
# 데이터 전처리
tmp_dir_list = directory_list
tmp_file_list = data_preprocess(file_list)
print(tmp_file_list)

['평택 설맞이 손 사진 공모전, "생명을 담다".', '2021 부산관광공사 시민 아이디어 공모전.', '제9회 아름다운 국토 사진 공모전.', '치매 예방 시니어. 경쟁.', '제4회 전국 중학교 가요 경연대회.', '제5회 GOBACK 예산시장 삼국축제 대회.', '제3회 시민참여대회.', '탄소중립성 달성을 위한 전력산업 아이디어 발굴 공모전입니다.', '2021 책방의 날 VLOG 공모전.', '2021 삼각산 동요 경연대회.', '오는 2021년에는 제6회 히말라야 사진영상 공모전이 연장된다.', '제주특별자치도 청소년 캐릭터 공모전.', '2021년 전국 학생 발명 상상력과 캐릭터 및 디자인 그리기 대회', '2021 군산시 현명한 가정생활 사진 공모전.', '제18회 청소년 자원봉사 실천사례 발표대회', '오일 데이터 활용 아이디어 공모전.', '2021년 SK하이닉스의 기후변화 위기 극복 공모전.', '제12회 통신서비스 이용자 주간통신서비스 피해예방 웹툰 공모전', '제1회 수목원·정원 국민참여 아이디어 공모전', '국민안전 119초 영상 공모전.', '2022 제주불축제 콘텐츠 아이디어 공모전.', '2021 한국기술교육대학교 안전혁신 아이디어 공모전', '증평인삼문화센터 BI 디자인 공모전.', '해외 송금 학생 할인 서비스 이름 공모전입니다.', '필환경 제주관광 슬로건 공모전.', '제5회 대학생 무역정책 토론회.', '다문화 이주여성 대상 임산물 로컬조리 UCC 공모전', '2021 대한민국 교육성취도 표창행사 경진대회', '청풍 아름 그린 사진 100초 영상 공모전.', '2021년 수출 핫 아이템 공모전']


In [28]:
print(len(origin_directory_list))
print(len(origin_file_list))

79
30


In [29]:
# 단순 유사도 검사
index_dict = simplest_classification(tmp_dir_list, tmp_file_list)
## update
result_dict.update(save_classificated_file(index_dict, origin_file_list, origin_directory_list))
tmp_file_list = rm_classificated_file(index_dict, tmp_file_list)
origin_file_list = rm_classificated_file(index_dict, origin_file_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: jpype._core.attachThreadToJVM is deprecated, use java.lang.Thread.attach instead
  import sys


In [30]:
# Word2Vec를 이용한 유사도 검사
index_dict = word2vec_similarity(tmp_dir_list, tmp_file_list)
## update
result_dict.update(save_classificated_file(index_dict, origin_file_list, origin_directory_list))
tmp_file_list = rm_classificated_file(index_dict, tmp_file_list)
origin_file_list = rm_classificated_file(index_dict, origin_file_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


In [31]:
# 자모단위 유사도 검사
index_dict = Phonology_classification(tmp_dir_list, tmp_file_list)
## update
result_dict.update(save_classificated_file(index_dict, origin_file_list, origin_directory_list))
tmp_file_list = rm_classificated_file(index_dict, tmp_file_list)
origin_file_list = rm_classificated_file(index_dict, origin_file_list)

In [32]:
# 기타 분류
result_dict.update(except_directory(origin_file_list))
result_list = dict_to_list(result_dict)

In [33]:
for i in result_list:
  print(i, end="\n")

사진/평택시 신중년의 손 사진 공모전 「인생을 담다」
아이디어/2021년도 부산관광공사 시민 아이디어 공모전
사진/제9회 아름다운 우리 국토 사진공모전
가요/제4회 전국 중학생 K-POP 경연대회
아이디어/탄소중립 달성을 위한 전력산업 아이디어 발굴 경진대회
동요/2021년 삼각산 동요대회
네이밍/2021년 나마스떼코리아 제6회 히말라야 사진(영상) 공모전 [기간연장]
캐릭터/제주특별자치도 청년 캐릭터 공모전
그리기/2021 전국 학생발명상상화 및 캐릭터 · 디자인 그리기 대회
사진/2021 군산시 슬기로운 가족생활 사진공모전
아이디어/석유 데이터 활용 아이디어 공모전
웹툰/제12회 통신서비스 이용자주간 통신서비스 피해예방 웹툰공모전
아이디어/제1회 수목원·정원 분야 국민참여 아이디어 공모전
아이디어/2022 제주들불축제 콘텐츠 아이디어 공모전
아이디어/2021년도 한국기술교육대학교 안전 분야 혁신 아이디어 공모전
디자인/증평인삼문화센터 BI 디자인 공모전
슬로건/필환경 제주관광 슬로건 공모전
사진/청풍아름그린 사진 & 100초영상 공모전
볼링/치매예방 시니어 가요 콩쿨
탁구 /제5회 GO BACK 예산장터 삼국축제 공모전
탁구 /제3회 시민참여 공모전-'쓰.담 공모전' 일터생활 공모전
오디션/<2021 서점의 날> 지역서점 브이로그 공모전
탁구 /제18회 청소년 봉사활동 실천사례 발표대회
오디션/2021 SK하이닉스 기후변화 위기극복 작품공모전
동영상/국민안전 119초 영상 공모전
오디션/모인 해외송금 학생 할인 서비스 네이밍 공모전
산업/제5회 대학생 통상정책 토론대회
컴퓨터/다문화 이주여성 대상 한국산 임산물 활용 현지 요리 UCC 공모전
탁구 /2021 대한민국 교육 공로 표창 행사(공모전)
캐릭터/2021 수출 핫템 공모전
